In [1]:
import sys
import os
from typing import List
from pprint import pprint
sys.path.insert(0, os.path.abspath('..'))

%load_ext autoreload
%autoreload 2

In [ ]:
from google.cloud import storage
from project_config import GCP_PROJECT_NAME

gcp_client = storage.Client(project=GCP_PROJECT_NAME)

In [3]:
import numpy as np
from rastervision.core.data import RasterioSource

from project_config import is_training, is_validation
from utils.schemas import ObservationPointer
from utils.data_management import observation_factory

## Proportion between mine and non-mine areas

In [4]:
from shapely.geometry import Polygon
import rasterio.features

In [ ]:

from utils.rastervision_pipeline import observation_to_scene
from experiment_configs.unet_fs_config import unet_orig_config as config

labels_train_raveled = []
labels_val_raveled = []
label_in_aoi_raveled = []
label_outside_aoi_raveled = []

for observation in observation_factory(gcp_client):
    is_train = is_training(observation.name)
    is_val = is_validation(observation.name)
    if not is_train and not is_val:
        print(f"Ignoring {observation.name}")
        continue

    scene = observation_to_scene(config, observation)
    label_arr = scene.label_source.get_label_arr()
    label_arr_raveled = label_arr.ravel()
    mask = rasterio.features.rasterize(scene.aoi_polygons, label_arr.shape)
    mask_raveled = mask.ravel()

    label_in_aoi_raveled.append(
        label_arr_raveled[mask_raveled != 0]
    )
    label_outside_aoi_raveled.append(
        label_arr_raveled[mask_raveled == 0]
    )
    
    if is_train:
        labels_train_raveled.append(label_arr_raveled)
    else:
        labels_val_raveled.append(label_arr_raveled)

all_labels_outside_aoi = np.hstack(label_outside_aoi_raveled)
all_labels_aoi = np.hstack(label_in_aoi_raveled)
all_labels_train = np.hstack(labels_train_raveled)
all_labels_val = np.hstack(labels_val_raveled)


In [17]:
from project_config import CLASS_CONFIG

class_mine_id = CLASS_CONFIG.get_class_id('sandmine')
class_nonmine_id = CLASS_CONFIG.get_class_id('other')

def calc_class_proportion(labels):
    mask_mine = (labels == class_mine_id)
    mask_nonmine = (labels == class_nonmine_id)

    count_mine = np.sum(mask_mine)
    count_nonemine = np.sum(mask_nonmine)
    count_total = len(labels)

    assert count_total == count_mine + count_nonemine
    mine_percentage = count_mine/count_total * 100
    nonmine_percentage = count_nonemine/count_total * 100
    return mine_percentage, count_mine

mine_percentage, _ = calc_class_proportion(np.hstack([all_labels_train, all_labels_val]))
print(f"Total dataset has {mine_percentage:.2f}%  mining area.")

mine_percentage, _ = calc_class_proportion(all_labels_train)
print(f"Training dataset has {mine_percentage:.2f}%  mining area.")

mine_percentage, _ = calc_class_proportion(all_labels_val)
print(f"Validation dataset has {mine_percentage:.2f}%  mining area.")

mine_percentage, _ = calc_class_proportion(all_labels_aoi)
print(f"Within AOIs, total dataset has {mine_percentage:.2f}%  mining area.")

mine_percentage, _ = calc_class_proportion(all_labels_outside_aoi)
print(f"Outside AOIs, total dataset has {mine_percentage:.2f}%  mining area.")

mine_percentage_per_observation = []
n_mine_pixels_per_observation = []
labels_full_dataset_raveled = [*labels_train_raveled, *labels_val_raveled]
for labels_of_observation in labels_full_dataset_raveled:
    mine_percentage_this_observation, n_mine_pixels_this_observation = calc_class_proportion(labels_of_observation)
    mine_percentage_per_observation.append(mine_percentage_this_observation)
    n_mine_pixels_per_observation.append(n_mine_pixels_this_observation)

print()
print(f"The median percentage of mine in an observation is {np.mean(mine_percentage_per_observation):.2f}%")
print(f"The median number of mine pixels in an observation is {np.mean(n_mine_pixels_per_observation):.0f}")

print()
n_total_pixels_per_observations = [len(labels_single_observation) for labels_single_observation in labels_full_dataset_raveled]
print(f"The median number pixels in an observation is {np.mean(n_total_pixels_per_observations):.0f}")


Total dataset has 2.69%  mining area.
Training dataset has 2.57%  mining area.
Validation dataset has 3.41%  mining area.
Within AOIs, total dataset has 6.07%  mining area.
Outside AOIs, total dataset has 0.07%  mining area.

The median percentage of mine in an observation is 3.19%
The median number of mine pixels in an observation is 33214

The median number pixels in an observation is 1236117


## Mean and Std of S1 images

In [ ]:
from rastervision.core.data.raster_transformer.nan_transformer import NanTransformer

all_observations: List[ObservationPointer] = list(observation_factory(gcp_client))

all_vv_raveled = []
all_vh_raveled = []
for observation in all_observations:
    raster_source = RasterioSource(
        observation.uri_to_s1,
        raster_transformers=[NanTransformer()]  # replaces NaNs with 0
    )

    vv_img = raster_source.get_image_array()[:,:,0]
    vh_img = raster_source.get_image_array()[:,:,1]
    all_vv_raveled.append(vv_img.ravel())
    all_vh_raveled.append(vh_img.ravel())
    
all_vv = np.hstack(all_vv_raveled)
all_vh = np.hstack(all_vh_raveled)


In [ ]:
print(f"VV: Mean = {np.mean(all_vv)}, Std = {np.std(all_vv)}")
print(f"VH: Mean = {np.mean(all_vh)}, Std = {np.std(all_vh)}")

## Area of observations

In [ ]:
from utils.data_management import get_location_from_key
from project_config import is_training, is_validation

dataset_summary = {}

observations_per_locations = {}
for observation in observation_factory(gcp_client):
    is_train = is_training(observation.name)
    is_val = is_validation(observation.name)
    if not is_train and not is_val:
        print(f"Ignoring {observation.name}")
        continue

    location = get_location_from_key(observation.name)
    if location in observations_per_locations:
        observations_per_locations[location].append(observation)
    else:
        observations_per_locations[location] = [observation]

total_area_km2 = 0
training_area_km = 0
validation_area_km = 0
smallest_area = 9999999
largest_area = 0
for location, observation_list in observations_per_locations.items():
    is_train = is_training(observation_list[0].name)
    is_val = is_validation(observation_list[0].name)

    # To determine the patch size, we only look into the first observations.
    # We expect that all observations cover the same geographical extent.
    raster_source = RasterioSource(observation_list[0].uri_to_s2, allow_streaming=False)
    coverage_area_km2 = raster_source.shape[0] * raster_source.shape[1] / 1e4  # Each pixel covers 100m^2
    summary_of_location = {
        "Number of observations": len(observation_list),
        "Patch size": raster_source.shape[:2],
        "Coverage area [km^2]": round(coverage_area_km2, 2),
        "Split": "TRAIN" if is_train else "VAL"
    }
    dataset_summary[location] = summary_of_location
    
    total_area_km2 += coverage_area_km2
    if is_train:
        training_area_km += coverage_area_km2
    if is_val:
        validation_area_km += coverage_area_km2

    smallest_area = min(smallest_area, coverage_area_km2)
    largest_area = max(largest_area, coverage_area_km2)


pprint(dataset_summary)
print(f"Total of {len(observations_per_locations)} locations")
print(f"Total area is {total_area_km2} km2")
print(f"Training area is {training_area_km} km2")
print(f"Validation area is {validation_area_km} km2")
print(f"Smallest location is {smallest_area} km2")
print(f"Largest location is {largest_area} km2")
